In [ ]:
!pip install pyspark

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder \
    .appName("Economic Data Analysis") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()

# Установка количества разделов для shuffle операций
spark.conf.set("spark.sql.shuffle.partitions", "50")

In [ ]:
# Чтение данных из HDFS
file_path = "hdfs://localhost:9000/user2/hadoop/economic_data/GDP.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Просмотр первых строк данных
df.show(5)

In [ ]:
pandas_df = df.toPandas()
pandas_df.head()

In [ ]:
# Отображение статистики числовых полей
summary_statistics = pandas_df.describe()

# Визуализация ВВП (GDP) по годам для каждой страны
plt.figure(figsize=(12, 8))
for country in pandas_df['Country'].unique():
    country_data = pandas_df[pandas_df['Country'] == country]
    plt.plot(country_data['Year'], country_data['GDP'], marker='o', label=country)

plt.title('GDP Over Years by Country')
plt.xlabel('Year')
plt.ylabel('GDP')
plt.legend()
plt.show()

summary_statistics

In [ ]:
pandas_df['Country'].unique()

In [ ]:
# Фильтрация данных только для Italy
Italy_data = pandas_df[pandas_df['Country'] == 'Italy']

# Отображение статистики для Italy
Italy_summary_statistics = Italy_data.describe()

# Визуализация ВВП (GDP) для Italy по годам
plt.figure(figsize=(10, 6))
plt.plot(Italy_data['Year'], Italy_data['GDP'], marker='o', linestyle='-', color='b', label='GDP')
plt.title('GDP of Italy Over Years')
plt.xlabel('Year')
plt.ylabel('GDP')
plt.legend()
plt.show()

Italy_summary_statistics

In [ ]:
# Преобразование Pandas DataFrame обратно в PySpark DataFrame
italy_data_spark = spark.createDataFrame(Italy_data)


In [ ]:
# Путь в HDFS для сохранения
file_path_hdfs = "hdfs://localhost:9000/user2/hadoop/economic_data/Italy_data.csv"

# Сохранение DataFrame в формате CSV в HDFS
italy_data_spark.write.csv(file_path_hdfs, header=True, mode='overwrite')

In [ ]:
#Ошибка org.apache.hadoop.security.AccessControlException: 
# Permission denied указывает на то, что текущий пользователь (devops) не имеет прав на запись в указанный каталог HDFS (/user2/hadoop/economic_data).

Изменение прав доступа к каталогу:

Используйте команду hdfs dfs -chmod для изменения прав доступа и разрешения записи для нужного пользователя:
hdfs dfs -chmod 777 /user2/hadoop/economic_data